In [85]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [86]:
abund_tb=pd.read_csv("/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/data/UNITE/BFTMS23071402 (single)OTU (Table).csv")
trim_tbl=pd.read_csv("/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/data/Trim_summary.csv")

In [87]:
abund_tb.head(5)

,ID,Name,Taxonomy,Combined Abundance,Min,Max,Mean,Median,Std,BFTMS23071402 Abundance,...,BFTMS23071409 Abundance,BFTMS23071410 Abundance,BFTMS23071411 Abundance,BFTMS23071412 Abundance,BFTMS23071413 Abundance,BFTMS23071414 Abundance,BFTMS23071415 Abundance,BFTMS23071416 Abundance,BFTMS23071417 Abundance,Sequence
0,Unidentified; SH489354.07FU_KP012839_reps_sing...,SH489354.07FU_KP012839_reps_singleton,Fungi; Basidiomycota; Agaricomycetes; Agarical...,2,0,1,0.1250,0.0,0.341565,1,...,0,0,0,0,0,0,0,0,0,TCAAAGATTCGATGATTCACGGAATTCTGCAATTCACATTACGTAT...
1,Kazachstania_aerobia; SH006357.07FU_AY582126_refs,SH006357.07FU_AY582126_refs,Fungi; Ascomycota; Saccharomycetes; Saccharomy...,26,0,8,1.6250,1.0,2.334524,1,...,0,2,0,0,0,2,1,0,8,TGAAGAACGCAGCGAAATGCGATACGTAATGTGAATTGCAGAATTC...
2,Saccharomyces_cerevisiae; SH478399.07FU_GQ2548...,SH478399.07FU_GQ254808_reps_singleton,Fungi; Ascomycota; Saccharomycetes; Saccharomy...,18,0,9,1.1250,0.0,2.247221,1,...,0,2,0,0,0,0,1,1,9,TGAAGAACGCAGCGAAATGCGATACGTAATGTGAATTGCAGAATTC...
3,Kazachstania_hellenica; SH023711.07FU_FJ810522...,SH023711.07FU_FJ810522_refs,Fungi; Ascomycota; Saccharomycetes; Saccharomy...,2,0,1,0.1250,0.0,0.341565,0,...,0,0,0,0,0,0,0,0,1,TGAAGAACGCAGCGAAATGCGATACGTAATGTGAATTGCAGAATTC...
4,Unidentified; SH629179.07FU_KY105124_reps_sing...,SH629179.07FU_KY105124_reps_singleton,Fungi; Ascomycota; Saccharomycetes; Saccharomy...,3,0,1,0.1875,0.0,0.403113,1,...,0,0,0,0,1,0,0,0,0,TTAAGAACATTGTTCGCCTAGACGCTCTCTTCTTATCGATAACGTT...


In [88]:
# Rename the columns by removing the keyword abundance and BFTMS
abund_tb.columns = [col.replace("Abundance", '') for col in abund_tb.columns]
abund_tb.columns = [col.replace(" ", '') for col in abund_tb.columns]
#abund_tb.columns = [col.replace("BFTMS", '') for col in abund_tb.columns]

##add a control column that gets values from the spcified control sample
control="BFTMS23071417"
#abund_tb["control"]=abund_tb[control]

samples=trim_tbl["Name"].astype(str).tolist()
#samples.append('control')

abund_tb['reference_lib']='UNITE'

In [89]:
## mark species with high risk of false positive detection in a column "false_pos_prone"  

# first split the ID column into species group name and species ID
abund_tb[['species group', 'species id']] = abund_tb['ID'].str.split(';', expand=True)
## create a new column "false_pos_prone" and mark species with high risk of false positive detection
abund_tb['false_pos_prone']='No'


def mark_species(df, ID_column, false_pos_tag, species_list):
    # Update Column B to True for rows where values in Column A start with any of the strings in the search list
    for name in species_list:
        df.loc[df[ID_column].str.startswith(name), false_pos_tag] = 'Yes'
    return df

false_pos_prone_spicies=['000000']
abund_tb = mark_species(abund_tb, 'species group', 'false_pos_prone', false_pos_prone_spicies)
#abund_tb.loc[abund_tb['species group'].contains(false_pos_prone_spicies), 'false_pos_prone'] = 'Yes'

In [90]:
## data normalization and corection for control
control="BFTMS23071417"
resid_val=0.00 # this value will be added to sample and control to avoid deviding to 0

# normalize all abundances based on the total reads for each sample
for samp in samples:
    ## read the trimmed reads from trim_tbl and get the value based on the sample name
    trimmed_reads=trim_tbl[trim_tbl["Name"]==samp]["Trimmed sequences"].values[0]
    abund_tb[samp+"_Normalized"]=abund_tb[samp]*trimmed_reads/1000000

## correct normized abundance by deducing control abundance
for samp in samples:
    abund_tb[samp+"_Normalized_corrected"]=abund_tb[samp+"_Normalized"]-abund_tb[control+"_Normalized"]
    ## calculate signal to noize 
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

## ## calculate signal to noize
for samp in samples:
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

In [91]:
## check one sample and compare it against the results from manual analysis
sample='BFTMS23071405'
sample_df=abund_tb[['ID', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']]
sample_df.sort_values(by=sample+'_Normalized_corrected', ascending=False)
sample_df.head(5)

,ID,BFTMS23071405,BFTMS23071417,BFTMS23071405_Normalized,BFTMS23071417_Normalized,BFTMS23071405_Normalized_corrected,BFTMS23071405_S:N
0,Unidentified; SH489354.07FU_KP012839_reps_sing...,0,0,0.000000,0.000000,0.000000,NaN
1,Kazachstania_aerobia; SH006357.07FU_AY582126_refs,2,8,0.346402,0.805624,-0.459222,-0.570020
2,Saccharomyces_cerevisiae; SH478399.07FU_GQ2548...,2,9,0.346402,0.906327,-0.559925,-0.617796
3,Kazachstania_hellenica; SH023711.07FU_FJ810522...,0,1,0.000000,0.100703,-0.100703,-1.000000
4,Unidentified; SH629179.07FU_KY105124_reps_sing...,0,0,0.000000,0.000000,0.000000,NaN


In [92]:
## UNITE strict

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    

    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=1000) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=10) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=5))
                            ]
    
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='UNITE_strict'
select_IDs.to_csv('/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/output_tables/UNITE_strict.csv')




In [93]:
## UNITE moderate

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    


    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=1000) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=7) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=3))
                            ]

    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='UNITE_moderate'
select_IDs.to_csv('/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/output_tables/UNITE_moderate.csv')




In [94]:
## LSU loose

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    

    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=1000) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=4) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=3)) &
                            (sample_abund['normalized_corrected']>=1.5))
                            ]

    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='UNITE_loose'
select_IDs.to_csv('/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/output_tables/UNITE_loose.csv')


